<h3><center><a href="https://indico.ectstar.eu/event/250/">ECT* DTP-TALENT 2025: Quantum Computing For Nuclear Physics</a></center></h3>

<h1><center>Quantum error mitigation</center></h1>

## Goal

The goal of this notebook is to implement [zero-noise extrapolation (ZNE)](https://mitiq.readthedocs.io/en/stable/guide/zne.html ([arXiv](https://arxiv.org/abs/1612.02058)), a particular [quantum error mitigation](https://mitiq.readthedocs.io/en/stable/guide/error-mitigation.html) technique, and use it to improve the performance of a simple quantum computation. By doing so, you will be able to implement ZNE in your own computations, both "from scratch" and by using existing software.

## Setup

In [ ]:
!pip install qiskit qiskit-aer qiskit-ibm-runtime pylatexenc --quiet

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import qiskit
import qiskit_aer
import qiskit_ibm_runtime

## The quantum computation

Let us evaluate the probability of the ground state in a circuit which logically compiles to identity. Throughout, let us call the circuit $|\psi\rangle$ and use $\Pi := |0\rangle \langle 0|$ so that the probability of the ground state is $\langle \psi | \Pi | \psi \rangle$.

In the cell below, write a function which returns a single-qubit quantum circuit with a specified number of `X` gates.

In [ ]:
"""Set up the quantum computation to use with ZNE."""
def get_circuit(depth: int = 50) -> qiskit.QuantumCircuit:
    """Returns a single-qubit circuit of `depth` X gates.

    Args:
        depth: Number of X gates in the circuit.
    """
    # -->  Your code here.

We'll use an even number of `X` gates so the final state of the circuit without any noise is $|0\rangle$. Let's visualize the circuit.

In [ ]:
"""Visualize the circuit."""
circuit = get_circuit(depth=50)
circuit.draw()

## The noisy result

It's possible to run this notebook on real devices if you have access to them, but that would take a while due to the fair-share queue. As an alternative below, you can use "fake backends" which model the noise characteristics of various devices. The next few cells show an example of how to use this.

In [ ]:
"""Getting a noisy device via qiskit.test.mock."""
# Get a noisy backend.
noisy_backend = qiskit_ibm_runtime.fake_provider.FakeSherbrooke()

# You can see some of its properties as follows.
print("Basis gates:", noisy_backend.configuration().basis_gates)
print("Number of qubits:", noisy_backend.configuration().num_qubits)

If you'd like, you can see "lower-level" properties with `noisy_backend.configuration().hamiltonian`. To run on this device, we do the following.

In [ ]:
"""Running on a noisy device via qiskit.test.mock."""
job = noisy_backend.run(circuit, shots=10_000)

Notes:
- If you're following along as intended, the circuit run in that cell doesn't have any measurements, so you'll get an error if you try to access them.

With ZNE, we need to run multiple circuits, so it's convenient to have a function for this. Complete the body of the function in the cell below by:

1. Measuring the qubit in the circuit if `measure` is `True`. You'll want to make a copy of the circuit so that you don't modify the input circuit.
2. Executing the circuit. **Be sure that all gates in the circuit are executed exactly as specified (i.e., that the compiler doesn't remove any gates).**
  - This is important because the circuits we want to run in ZNE have structures like $G G^\dagger G$. A compiler may recognize this and simplify to $G$. But in ZNE we want all gates to be implemented on the device to scale the noise.
3. Returning the ratio of `0`s measured to the total number of shots (i.e., the expectation value of our observable).

In [ ]:
def execute(
    circuit: qiskit.QuantumCircuit,
    backend: "qiskit.Sampler" = qiskit_ibm_runtime.fake_provider.FakeSherbrooke(),
    shots: int = 10_000,
    measure: bool = True,
) -> float:
    """Returns the ground state probability of the circuit run on the backend.

    Args:
        circuit: Single-qubit circuit to estimate the ground state probability of.
        backend: Backend to run the circuit on.
        shots: Number of times to sample from the circuit.
        measure: If True, measure all qubits in the circuit, else do nothing.
    """
    # -->  Your code here.

Using this function, we can run the quantum computation and see our result.

In [ ]:
noisy_value = execute(circuit)
print("Noisy: \t⟨𝜓|Π|𝜓⟩ =", noisy_value)

## The true (noiseless) result

For testing how well ZNE works, it's convenient to know what the "true" or noiseless expectation value is. We already know the true result for our simple example, but we can do this in general for simple cases by using our function with a noiseless simulator as below.

In [ ]:
"""Determine the true (noiseless) value by running on a noiseless backend."""
true_value = execute(circuit, qiskit_aer.AerSimulator())
print("True: \t⟨𝜓|Π|𝜓⟩ =", true_value)

## Implementing zero noise extrapolation

Now we implement ZNE in three steps.

### Step 1 / 3: Scale the noise in the computation

First we need to scale the noise in the computation. As described in lecture, there are various ways to do this. We will use the simplest gate-level method to scale noise. Letting $U$ denote the complete circuit (without measurements), map $U$ to $U U^\dagger U$. The next cell shows how to do this in Qiskit.

In [ ]:
"""Map U to U U^dag U."""
inverse = circuit.inverse()
scaled_circuit = circuit.compose(inverse).compose(circuit)

You may wish to print / inspect this circuit.

For ZNE, we want to run at several noise levels. This corresponds to the mapping $U \mapsto U (U^\dagger U)^k$ for $k = 1, 2, 3, ...$. In the next cell, you are asked to complete the body of a function to do this.

In [ ]:
def scale_circuit(circuit: qiskit.QuantumCircuit, scale_factor: int) -> qiskit.QuantumCircuit:
    """Returns U (U^dag U)^k where U is the input circuit and k is the scale_factor.

    Args:
        circuit: Circuit to scale. Should not have measurements.
        scale_factor: The integer k in the above formula (how many times to add U^dag U).
    """
    # -->  Your code here.

One step down, and that's the hardest one!

### Step 2 / 3: Execute the noise-scaled computations

For this next step, we need to run our noise-scaled circuits. Because we defined an `execute` function above to compute our expectation value of interest, this is as easy as calling `execute` on each scaled circuit, as shown in the next cell.

In [ ]:
"""Pick scale factors and execute the computation at different noise levels."""
# Pick scale factors.
scale_factors = [1, 2, 3]

# Scale the circuits at each scale factor.
scaled_circuits = [scale_circuit(circuit, scale_factor) for scale_factor in scale_factors]

# Run the noise-scaled circuits.
expectation_values = [execute(scaled_circuit) for scaled_circuit in scaled_circuits]

In [ ]:
"""Plot the expectation value vs. scale factor data."""
# -->  Your code here.

### Step 3 / 3: Fit a model to the noise-scaled expectation values

If you kept the intended `depth` and `scale_factors`, the above trend line should look linear. So, let's fit a line $E(\lambda) = a + b \lambda$ to the data. We can do this very easily with `np.polyfit` as shown below.

In [ ]:
"""Fit a model to the data."""
best_fit_coeffs = np.polyfit(scale_factors, expectation_values, deg=1)

Now we have best-fit coefficients $E(\lambda) = a^* + b^* \lambda$. Use this to get the zero-noise $(\lambda = 0)$ value in the cell below.

In [ ]:
"""Get the `zne_value` from the best-fit coefficients found above."""
zne_value = 0.0  # <--  Your code here.

Congrats! you just implemented zero-noise extrapolation.

### Compare the results

Let's see how ZNE performed.

In [ ]:
"""Display the results."""
print(f"True: \t⟨𝜓|Π|𝜓⟩ = {true_value:.3f}")
print(f"Noisy \t⟨𝜓|Π|𝜓⟩ = {noisy_value:.3f}")
print(f"ZNE: \t⟨𝜓|Π|𝜓⟩ = {zne_value:.3f}")

# Compute the "quantum error mitigation (QEM) factor.
error_mitigation_factor = (true_value - noisy_value) / (true_value - zne_value)
print(f"\nQEM improvement factor: {error_mitigation_factor:.3f}")

### To consider

- How do your results change if you fit a different degree polynomial?
- How do your results change if you use more / different `scale_factors`?
- How do your results change if you use a lower / higher depth circuit?
- Try fitting the data with an expononential or similar function using [SciPy curve fit](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html).
- The fit you "should" use ideally comes from some knowledge of how the computation should look at different scale factors, based on both the details of the circuit, expectation value, and noise model of the device.

## Software for error mitigation

As we've (hopefully) seen, implementing zero-noise extrapolation isn't terribly difficult. But we just skimmed the surface - there's different ways to scale noise (including fractionally), choose scale factors (including adaptively), and fit models to the data. Error mitigation software packages like [Mitiq](https://mitiq.readthedocs.io/en/stable/) and [Qermit](https://cqcl.github.io/Qermit/) handle these details, as well as implement other error mitigation techniques. You may wish to use them for your own experiments instead of implementing everything "from scratch".

## Challenge

Done early?

- Try to implement zero-noise extrapolation for a quantum computation in one of the hands-on sessions at this school.
- Reproduce a zero-noise extrapolation experiment in literature, for example [this one increasing effective quantum volume](https://github.com/unitaryfund/mitiq-qv/blob/master/main.ipynb) or this one arguing the [evidence of utility of quantum computing before fault tolerance](https://www.nature.com/articles/s41586-023-06096-3).
- Check out the other error mitigation methods, such as probabilistic error cancellation, in [Mitiq](https://mitiq.readthedocs.io/en/stable/) and/or [Qermit](https://cqcl.github.io/Qermit/).